# GSM-VEVP: visco-elasto-visco-plasticity with isotropic hardening

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from IPython.display import display, Math, Markdown
from bmcs_matmod.api import GSM1D_VEVP
gsm = GSM1D_VEVP()
display(Markdown(gsm.latex_potentials()))

## Linearly ascending loading

In [ ]:
E_val = 30e+3  # Young's modulus in MPa
K_val = 1e+3  # Bulk modulus in MPa
f_c_val = 15 # Strength in MPa
eta_vp_val = 10e+4  # Viscosity in MPa s
eta_ve_val = 10e+4  # Viscosity in MPa s
mparams = dict(E=E_val, K=K_val, f_c=f_c_val, eta_ve=eta_ve_val, eta_vp=eta_vp_val)

f_time = 10  # Final time in secondsfinal_time = 1
n_t = 151
t_t = np.linspace(0, f_time, n_t)
eps_max = 0.0025
eps_n_t = np.linspace(0, eps_max, n_t)

response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t_ec, eps_ta_ec, sig_ta_ec, Eps_t_ec, Sig_t_ec, iter_t_ec, lam_t_ec, (d_t_t_ec, d_eps_ta_ec) = response
eps_t_ec = eps_ta_ec[:, 0]
sig_t_ec = sig_ta_ec[:, 0, 0]
# use the stress history to reproduce the response using stress control
response = gsm.get_G_response(sig_t_ec, t_t, **mparams)
t_t_sc, eps_ta_sc, sig_ta_sc, Eps_t_sc, Sig_t_sc, iter_t_sc, lam_t_sc, (d_t_t_sc, d_eps_ta_sc) = response
iter_t = iter_t_sc[:, 0]
eps_t_sc = eps_ta_sc[:, 0, 0]
sig_t_sc = sig_ta_sc[:, 0]
eps_t_ec.shape, sig_t_ec.shape, eps_t_sc.shape, sig_t_sc.shape

In [ ]:
eps_ve_t_ec, eps_vp_t_ec, z_ve_t_ec = Eps_t_ec[:, 0, :].T
sig_ve_t_ec, sig_vp_t_ec, Z_ve_t_ec = Sig_t_ec[:, 0, :].T
eps_ve_t_sc, eps_vp_t_sc, z_ve_t_sc = Eps_t_sc[:, 0, :].T
sig_ve_t_sc, sig_vp_t_sc, Z_ve_t_sc = Sig_t_sc[:, 0, :].T
(sig_ve_t_ec.shape, eps_ve_t_ec.shape, sig_ve_t_sc.shape, eps_ve_t_sc.shape)

In [ ]:
fig, (ax_sig, ax_eps) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain curves (force vs strain)
ax_sig.plot(eps_t_ec, sig_t_ec, color='cadetblue', lw=5, label=r'$\varepsilon$')
ax_sig.plot(eps_t_ec - eps_ve_t_ec, sig_t_ec, ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.plot(eps_t_ec - eps_vp_t_ec, sig_t_ec, ls='dashdot', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{vp}$')

ax_sig.plot(eps_t_sc, sig_t_sc, color='red', label=r'$\varepsilon$')
ax_sig.plot(eps_t_sc - eps_ve_t_sc, sig_t_sc, ls='dashed', color='red', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.plot(eps_t_sc - eps_vp_t_sc, sig_t_sc, ls='dashdot', color='red', label=r'$\varepsilon-\varepsilon^\mathrm{vp}$')

ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

# Plot strain histories vs. time
ax_eps.plot(t_t_ec, eps_t_ec, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t_ec, eps_ve_t_ec, label=r'$\varepsilon_\mathrm{ve}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.plot(t_t_ec, eps_vp_t_ec, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashdot', lw=1)

ax_eps.plot(t_t_sc, eps_t_sc, label=r'$\varepsilon$', color='red', lw=1)
ax_eps.plot(t_t_sc, eps_ve_t_sc, label=r'$\varepsilon_\mathrm{ve}$', color='red', ls='dashed', lw=1)
ax_eps.plot(t_t_sc, eps_vp_t_sc, label=r'$\varepsilon_\mathrm{vp}$', color='red', ls='dashdot', lw=1)

ax_eps.legend()
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

plt.tight_layout()
plt.show()